# 实验五

崔士强 PB22151743

In [158]:
import pandas as pd
import numpy as np

## 实验三数据

下面使用实验三的数据，对`diagnosis`进行预测。

### 数据读取及预处理

In [159]:
df = pd.read_csv('data.csv', encoding='utf-8')

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          568 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   567 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               567 non-null    float64
 14  perimeter_se             5

In [160]:
df = df.dropna()
df = df.reset_index(drop=True)

### 主实验

对于主实验，采取下面两种算法

1. 支持向量机（SVM）

SVM的目的是找到一个超平面，它可以有效地分离不同类别的数据点，同时尽可能地最大化不同类别之间的间隔。SVM对于小到中等数据集表现良好，尤其是在数据维度较高时。

参考文献：

	•	Cortes, C., & Vapnik, V. (1995). Support-vector networks. Machine Learning, 20(3), 273-297.

1. 梯度提升树（Gradient Boosting Machines, GBM）

GBM通过逐步修正前一个模型的残差来增强模型的预测能力，在处理各种统计分类和回归问题时表现出色。GBM特别适用于处理复杂的非线性关系。

参考文献：

	•	Friedman, J. H. (2001). Greedy function approximation: A gradient boosting machine. Annals of statistics, 1189-1232.

In [161]:
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [162]:
# 分离特征和目标变量
X = df.drop(['diagnosis', 'id'], axis=1)  # 特征
y = df['diagnosis']  # 目标变量

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化模型
svm_model = SVC(kernel='linear', random_state=42)  # 线性核SVM
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# 训练模型
svm_model.fit(X_train, y_train)
gbm_model.fit(X_train, y_train)

# 预测和评估
svm_pred = svm_model.predict(X_test)
gbm_pred = gbm_model.predict(X_test)

print("SVM Classification Report:")
print(classification_report(y_test, svm_pred))

print("GBM Classification Report:")
print(classification_report(y_test, gbm_pred))

SVM Classification Report:
              precision    recall  f1-score   support

           B       0.94      0.98      0.96        63
           M       0.98      0.92      0.95        49

    accuracy                           0.96       112
   macro avg       0.96      0.95      0.95       112
weighted avg       0.96      0.96      0.96       112

GBM Classification Report:
              precision    recall  f1-score   support

           B       0.94      0.95      0.94        63
           M       0.94      0.92      0.93        49

    accuracy                           0.94       112
   macro avg       0.94      0.94      0.94       112
weighted avg       0.94      0.94      0.94       112



### 参数优化

In [163]:
from sklearn.model_selection import GridSearchCV

In [164]:
# 设置参数网格
svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

gbm_params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

# 创建 GridSearchCV 对象
svm_grid = GridSearchCV(SVC(random_state=42), svm_params, cv=5, scoring='accuracy')
gbm_grid = GridSearchCV(GradientBoostingClassifier(random_state=42), gbm_params, cv=5, scoring='accuracy')

# 执行网格搜索
svm_grid.fit(X_train, y_train)
gbm_grid.fit(X_train, y_train)

# 最佳参数和最佳模型评分
print("Best SVM Parameters:", svm_grid.best_params_)
print("Best SVM Score:", svm_grid.best_score_)
print("Best GBM Parameters:", gbm_grid.best_params_)
print("Best GBM Score:", gbm_grid.best_score_)

# 使用最佳参数模型进行预测
svm_best = svm_grid.best_estimator_.predict(X_test)
gbm_best = gbm_grid.best_estimator_.predict(X_test)

# 评估
print("SVM Test Set Performance:")
print(classification_report(y_test, svm_best))

print("GBM Test Set Performance:")
print(classification_report(y_test, gbm_best))

Best SVM Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best SVM Score: 0.9620224719101124
Best GBM Parameters: {'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 100}
Best GBM Score: 0.966541822721598
SVM Test Set Performance:
              precision    recall  f1-score   support

           B       0.94      1.00      0.97        63
           M       1.00      0.92      0.96        49

    accuracy                           0.96       112
   macro avg       0.97      0.96      0.96       112
weighted avg       0.97      0.96      0.96       112

GBM Test Set Performance:
              precision    recall  f1-score   support

           B       0.94      0.95      0.94        63
           M       0.94      0.92      0.93        49

    accuracy                           0.94       112
   macro avg       0.94      0.94      0.94       112
weighted avg       0.94      0.94      0.94       112



与之前的结果基本一致。

## 实验四数据

下面使用实验四的数据，对`Class`进行预测。

### 数据读取及预处理

In [165]:
df = pd.read_csv('data2.csv')

对数据进行预处理，过程包括去除缺失值，转换为独热向量等

In [166]:
columns_with_missing_values = df.columns[df.isnull().any()].tolist()

print("Columns with missing values:")
print(columns_with_missing_values)

df = df.dropna()

print("Value distribution for 'tumor-size' before corrections:")
print(df['tumor-size'].value_counts())

print("\nValue distribution for 'inv-nodes' before corrections:")
print(df['inv-nodes'].value_counts())

correction_map_tumor_size = {
    '14-Oct' : '10-14',
    '9-May' : '5-9'
}

correction_map_inv_nodes = {
    '5-Mar' : '3-5',
    '8-Jun' : '6-8',
    '11-Sep' : '9-11',
    '14-Dec' : '12-14'
}

df['tumor-size'] = df['tumor-size'].replace(correction_map_tumor_size)
df['inv-nodes'] = df['inv-nodes'].replace(correction_map_inv_nodes)

print("\nCorrected value distribution for 'tumor-size':")
print(df['tumor-size'].value_counts())

print("\nCorrected value distribution for 'inv-nodes':")
print(df['inv-nodes'].value_counts())


variables_df = pd.read_excel('variables.xlsx')

ind2val = {}
current_index = 0

for i, row in variables_df.iterrows():
    variable_name = row['Variable Name']
    values = row['Description'].split(', ')
    for value in values:
        ind2val[current_index] = f"{variable_name}={value}"
        current_index += 1

val2ind = {v: k for k, v in ind2val.items()}

for column in df.columns:
    if column in variables_df['Variable Name'].values:
        # 为每个列值构建一个映射函数
        df[column] = df[column].apply(lambda x: val2ind[f"{column}={x}"])

# 打印ind2val字典
for key in list(ind2val.keys()):
    print(key, ind2val[key])

print(df)

Columns with missing values:
['node-caps', 'breast-quad']
Value distribution for 'tumor-size' before corrections:
tumor-size
30-34     57
25-29     51
20-24     48
15-19     29
14-Oct    28
40-44     22
35-39     19
0-4        8
50-54      8
9-May      4
45-49      3
Name: count, dtype: int64

Value distribution for 'inv-nodes' before corrections:
inv-nodes
0-2       209
5-Mar      34
8-Jun      17
11-Sep      7
15-17       6
14-Dec      3
24-26       1
Name: count, dtype: int64

Corrected value distribution for 'tumor-size':
tumor-size
30-34    57
25-29    51
20-24    48
15-19    29
10-14    28
40-44    22
35-39    19
0-4       8
50-54     8
5-9       4
45-49     3
Name: count, dtype: int64

Corrected value distribution for 'inv-nodes':
inv-nodes
0-2      209
3-5       34
6-8       17
9-11       7
15-17      6
12-14      3
24-26      1
Name: count, dtype: int64
0 Class=no-recurrence-events
1 Class=recurrence-events
2 age=10-19
3 age=20-29
4 age=30-39
5 age=40-49
6 age=50-59
7 age=60-6

### 主实验

我们选择两种算法模型进行比较：

	• 逻辑回归：广泛用于二分类问题的线性模型，它预测的是观察属于特定类别的概率。参考资料: Hosmer, D. W., & Lemeshow, S. (2000). Applied Logistic Regression.
	• 随机森林：一个基于决策树的集成学习算法，通过构建多棵树并取它们的多数投票来提高预测准确性和稳定性。参考资料: Breiman, L. (2001). Random Forests. Machine Learning.

我们使用所有可用特征，因为逻辑回归和随机森林能够较好地处理高维数据，并从中选择重要的特征。

在这一步，第一次尝试的结果是准确度达到100%（在这里发现的原因是我先做了实验四数据），更换多种方法及模型均无果。最终发现原因是没有把id从特征中去除。

In [167]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [168]:
# 分离特征和目标变量
X = df.drop(['Class', 'id'], axis=1)  # 特征
y = df['Class']  # 目标变量

# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化模型
logreg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练逻辑回归
logreg_pipeline = Pipeline(steps=[('classifier', logreg)])
logreg_pipeline.fit(X_train, y_train)
y_pred_logreg = logreg_pipeline.predict(X_test)

# 训练随机森林
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# 打印逻辑回归模型的评估结果
print("逻辑回归模型评估结果:")
print(classification_report(y_test, y_pred_logreg))

# 打印随机森林模型的评估结果
print("随机森林模型评估结果:")
print(classification_report(y_test, y_pred_rf))

逻辑回归模型评估结果:
              precision    recall  f1-score   support

           0       0.73      0.95      0.82        37
           1       0.75      0.32      0.44        19

    accuracy                           0.73        56
   macro avg       0.74      0.63      0.63        56
weighted avg       0.74      0.73      0.69        56

随机森林模型评估结果:
              precision    recall  f1-score   support

           0       0.69      0.78      0.73        37
           1       0.43      0.32      0.36        19

    accuracy                           0.62        56
   macro avg       0.56      0.55      0.55        56
weighted avg       0.60      0.62      0.61        56



尝试使用k折交叉验证，结果如下

In [169]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# 分离特征和目标变量
X = df.drop(['Class', 'id'], axis=1)  # 特征
y = df['Class']  # 目标变量

# 初始化模型
logreg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 定义交叉验证策略
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 定义评估指标
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

# 执行逻辑回归的交叉验证
logreg_scores = cross_validate(logreg, X, y, cv=cv_strategy, scoring=scoring_metrics)
print("逻辑回归交叉验证评估结果:")
for metric, scores in logreg_scores.items():
    print(f"{metric}: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})")

# 执行随机森林的交叉验证
rf_scores = cross_validate(rf, X, y, cv=cv_strategy, scoring=scoring_metrics)
print("\n随机森林交叉验证评估结果:")
for metric, scores in rf_scores.items():
    print(f"{metric}: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})")

逻辑回归交叉验证评估结果:
fit_time: 0.006 (+/- 0.009)
score_time: 0.002 (+/- 0.000)
test_accuracy: 0.736 (+/- 0.039)
test_precision: 0.720 (+/- 0.044)
test_recall: 0.736 (+/- 0.039)
test_f1: 0.702 (+/- 0.039)

随机森林交叉验证评估结果:
fit_time: 0.041 (+/- 0.001)
score_time: 0.005 (+/- 0.001)
test_accuracy: 0.736 (+/- 0.027)
test_precision: 0.717 (+/- 0.040)
test_recall: 0.736 (+/- 0.027)
test_f1: 0.715 (+/- 0.049)


### 参数优化

In [170]:
# 参数网格搜索
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None]
}
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# 使用最佳模型进行预测和评估
best_params = grid_search.best_params_
best_score = grid_search.best_score_
y_pred_best_rf = grid_search.predict(X_test)
best_rf_scores = classification_report(y_test, y_pred_best_rf, output_dict=True)

# 打印最佳参数和最佳分数
print("最佳参数组合:", best_params)
print("最佳交叉验证分数:", best_score)

# 打印使用最佳模型的评估结果
print("最佳模型评估结果:")
print(classification_report(y_test, y_pred_best_rf))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
最佳参数组合: {'max_depth': 10, 'n_estimators': 50}
最佳交叉验证分数: 0.7919289152165865
最佳模型评估结果:
              precision    recall  f1-score   support

           0       0.69      0.84      0.76        37
           1       0.45      0.26      0.33        19

    accuracy                           0.64        56
   macro avg       0.57      0.55      0.54        56
weighted avg       0.61      0.64      0.61        56



略有提升。